This interactive notebook will help you process the data from the hydrogen spectrum lab. You are however welcome to use your own method. You can run this notebook both on your computer (You will need to have python 3 with used packages) or you can use services such as google-colab. Start by running the following two cells (shift+enter).

In [1]:
# !pip install uncertainties==3.1.7  # include this line if you want to work in google colab

import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as c
from scipy.stats import t as student_t
import pandas as pd
import statistics
from uncertainties import ufloat

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
lyman_series   = 1/1**2 - 1/np.array(np.arange(2,10))**2
balmer_series  = 1/2**2 - 1/np.array(np.arange(3,10))**2
paschen_series = 1/3**2 - 1/np.array(np.arange(4,10))**2

# lyman_wavelengths_nm = (rydberg * lyman_series)**(-1) / 1e-9
# balmer_wavelengths_nm = (rydberg * balmer_series)**(-1) / 1e-9
# paschen_wavelengths_nm = (rydberg * paschen_series)**(-1) / 1e-9

fill in the measured wavelengths and run the following cell (shift+enter)

In [9]:
measured_wavelengths_nm = np.array([393.8, 410.5, 434.9, 487.0, 580.0, 618.6, 656.6, 779.1, 848.2])

Now run the following cell and play with the sliders. The black lines are the spectral lines you measured. If you move the sliders, eventually colored lines will show up. Red lines correspond to the Lyman series, green to the Balmer series and blue to the Paschen series. Try to match the colored lines to the lines you measured and find the Rydberg's constant. The colored line follow 
$$
\lambda = \frac{1}{R \, \left( n_i^{-2} - n_f^{-2}\right)}
$$

In [10]:
rydberg_constant_slider_mantissa = widgets.FloatLogSlider(
    value=0.66468,
    base=10,
    min=-0.5, # min exponent of base
    max=0.5, # max exponent of base
    step=0.000001, # exponent step
    description='R =',
    readout_format='.4f'
)

rydberg_constant_slider_exponent = widgets.IntSlider(
    value=0,
    min=-10,
    max=10,
    step=1,
    description='x 10^',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

box = widgets.HBox([rydberg_constant_slider_mantissa, rydberg_constant_slider_exponent])


rydberg_constant_m = rydberg_constant_slider_mantissa.value * 10**rydberg_constant_slider_exponent.value


def plot_things(
    rydb_mantissa,
    rydb_exponent
):
    # Clear the current plot
    fig,ax = plt.subplots(figsize=(7,1))
    

    rydb_m = rydb_mantissa * 10**rydb_exponent 
    
    for wl in measured_wavelengths_nm:
        plt.plot([wl,wl], [0,1] , color ='black')

    for bc in balmer_series:
        wl = 1 / (rydb_m * bc / 1e9)
        plt.plot([wl,wl], [0,1] , color ='g')
    
    for bc in lyman_series:
        wl = 1 / (rydb_m * bc / 1e9)
        plt.plot([wl,wl], [0,1] , color ='r')

    for bc in paschen_series:
        wl = 1 / (rydb_m * bc / 1e9)
        plt.plot([wl,wl], [0,1] , color ='b')

    ax.set_title(rf"Rydberg's constant   $R = {rydb_mantissa:.4f} \times 10^{{{rydb_exponent}}} $ m$^{{-1}}$")
    ax.set_xlabel('wavelength (nm)')
    ax.set_xlim(350,900)
    ax.set_ylim(0, 1)
    ax.set_yticks([])
    ax.grid()
    # ax.set_aspect(1)
    

interactive_plot = interactive(
    plot_things,
    rydb_mantissa = rydberg_constant_slider_mantissa,
    rydb_exponent = rydberg_constant_slider_exponent
).children[-1]
    
display(box)
display(interactive_plot)

Output()

If you succesfully matched the colored lines onto the lines you measured, you can continue. From the plot above you can match the individual wavelengths $\lambda$ with the corresponding quantum numbers of the transitions $n_i$ and $n_f$.
Fill in the missing values and code (labeled with #*) in the following cell and run it so we can estimate the Rydberg's constant.

In [18]:
wavelengths_nm = np.array([393.8,410.5, 434.9, 487.0, 656.6])
#wavelengths_nm = np.flip(wavelengths_nm)
n_i = np.array( [7,6,5,4,3] , dtype=float)
n_f = np.array( [2,2,2,2,2], dtype=float)
rydberg_est_m = 1/((wavelengths_nm*1e-9)*((1/n_f**2) - (1/n_i**2)))

rydberg_constant_tabular = 1.0967e7              # 1/m

df = pd.DataFrame({
    'wavelengths (nm)': wavelengths_nm,
    'n_i': n_i,
    'n_f': n_f,
    'R (m^-1)': rydberg_est_m
})

print(df)

mean_rydberg_est_m = statistics.mean(rydberg_est_m)
stdev_rydber_est_m = statistics.stdev(rydberg_est_m)
student_coefficient = student_t.ppf(1 - 0.05 / 2, len(rydberg_est_m)-1)
error = stdev_rydber_est_m * student_coefficient

rydberg_const_estimate = ufloat(mean_rydberg_est_m, error)

print(f'Estimated Rydberg constant is: {rydberg_const_estimate} m^-1')
print(f'Tabular Rydberg constant is: {rydberg_constant_tabular} m^-1')
print(f"Absolute difference of measured and tabular R is: {rydberg_const_estimate-rydberg_constant_tabular} m^-1'")


   wavelengths (nm)  n_i  n_f      R (m^-1)
0             393.8  7.0  2.0  1.106032e+07
1             410.5  6.0  2.0  1.096224e+07
2             434.9  5.0  2.0  1.094942e+07
3             487.0  4.0  2.0  1.095140e+07
4             656.6  3.0  2.0  1.096558e+07
Estimated Rydberg constant is: (1.098+/-0.013)e+07 m^-1
Tabular Rydberg constant is: 10967000.0 m^-1
Absolute difference of measured and tabular R is: (0.1+/-1.3)e+05 m^-1'


How do your results compare to the tabulated? What other constants could you find from this measurement? 